In [ ]:
!pip install PyMuPDF rdflib textacy #pyldavis

     |████████████████████████████████| 8.7 MB 31.8 MB/s 
     |████████████████████████████████| 407 kB 45.4 MB/s 
     |████████████████████████████████| 200 kB 49.2 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
     |████████████████████████████████| 6.0 MB 36.6 MB/s 
     |████████████████████████████████| 137 kB 55.1 MB/s 
     |████████████████████████████████| 481 kB 41.1 MB/s 
     |████████████████████████████████| 2.0 MB 43.2 MB/s 
     |████████████████████████████████| 628 kB 55.0 MB/s 
     |████████████████████████████████| 10.1 MB 67.2 MB/s 
     |████████████████████████████████| 181 kB 61.6 MB/s 
     |████████████████████████████████| 451 kB 48.5 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1231110 sha256=b0b3464aecaeb7d3ddac0d8ad50de3cc7f76e0e9e39348716ed888dce19d1a49
  Stored in directory: /root/.cache/pip/wheels/38/70/71/ca13ea3d36ccd0b3d0ec7d7a4ca

In [ ]:
!pip install --upgrade spacy

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.4 MB 1.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
cd /content/drive/MyDrive/dataset/ai_cognition/

/content/drive/MyDrive/dataset/ai_cognition


In [ ]:
import spacy
#import benepar

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
nlp = spacy.load("en_core_web_lg")

# Classes

## ProcessText

In [ ]:
import re

class ProcessText:
    
    @staticmethod
    def clean_text(sentence:str):
        text = ProcessText.remove_special_char(ProcessText.extract_only_text(sentence))
        return ProcessText.remove_double_space(text)
    
    def remove_double_space(text):
        return re.sub(r'([\s])\1{1,}|([\.])\1{1,}', r'\1', text)

    @staticmethod
    def extract_only_text(text):
        """ Keep only words and special characters from a text and

        Args:
            text (str): any text 

        Returns:
            str: filtered text
        """
        text = re.sub('\S*@\S*\s?', '', text)
        text = re.sub('\S*www\S*\s?', '', text)
        text = re.sub('\S*http\S*\s?', '', text)
        text = re.sub('\S*[.][a-zA-Z]{2,4}', '', text)
        text = re.sub('[0-9]+ - PROJECT RISK MANAGEMENT','',str(text))
        #text = re.sub('(Figure|Table) [0-9]+(\s)*-?(\s)*[0-9]+','',str(text))
        #text = re.sub('Section ([0-9].)*([0-9])*','',str(text))
        #removing chapter 
        text = re.sub('CHAPTER .*','',str(text))
        # removing titles and subtitles
        text = re.sub('(\d+\.)+\d*.*','',str(text))
        text = re.sub('\d+','',str(text))
        # removing new line characters
        #text = re.sub('\n',' ',str(text))
        #text = re.sub('\n ','',str(text))
        #tabulations
        text = re.sub('\t',' ',str(text))
        # removing hyphens
        #text = re.sub("-",' ',str(text))
        #text = re.sub("—",' ',str(text))
        # removing quotation marks
        text = re.sub('\"','',str(text))
        # removing any reference to outside text
        #text = re.sub("[\(\[].*?[\)\]]", "", str(text))
        
        return text
    
    def remove_special_char(text:str):
        """Extract special characters from a text and keep only ' and ’ and “” 

        Args:
            text (str): any text

        Returns:
            str: filtered text
        """
        return text #re.sub(r"[^a-zA-Z0-9\-“”\s\'\.]","",text)

## ParsePDF

In [ ]:
import fitz
import unicodedata

class ParsePDF():
    """This class provides methodes to parse a power point file
    """

    def __init__(self,filename):
        """constructor

            Args:
                filename (str): path to the file
        """
        try:
            self._f = fitz.open(filename)
        except:
            raise FileNotFoundError(f"can't open file {filename}")

    @property
    def number_pages(self):
        return len(self._f)

    def content(self,start=0,end=0):
        
        text = ""
        if start > self.number_pages or end > self.number_pages or start > end :
            return text

        for page in self._f.pages(start-1, end+1, 1):
            text += re.sub(r'([\s])\1{1,}|([\.])\1{1,}', r'\1', page.get_text())
        return unicodedata.normalize("NFKD",text) 

        
            

## Evaluator

In [ ]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis

class Evaluator:

    def computeTFIDF(self, documents):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(documents)
        feature_names = vectorizer.get_feature_names()
        dense = vectors.todense()
        denselist = dense.tolist()
        df = pd.DataFrame(denselist, columns=feature_names)
        df.index = ([f"TFIDF_{i}" for i in range(1,len(documents)+1)])

        return df.T 

    def similarity(self,documents):
        similarities = {}
        for i,document in enumerate(documents,1):
            doc = nlp(document)
            sim = []
            for document_ in documents:
                sim.append(doc.similarity(nlp(document_))) 
            similarities[f"Doc_{i}"] = sim
        df = pd.DataFrame(similarities, index=[f"Doc_{i}" for i in range(1,len(documents)+1)])
        return df

    def display_lda(self,lda_model, corpus, id2word):
        return
        pyLDAvis.enable_notebook()
        vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
        return vis

    def LDA(self,data_words):
        id2word = corpora.Dictionary(data_words)
        corpus = []
        
        for text in data_words:
            new = id2word.doc2bow(text)
            corpus.append(new)
        num_topic = len(corpus)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topic,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           eta = [.01]*len(id2word.keys()),
                                           alpha=[.01]*num_topic)

        

        return lda_model ,self.display_lda(lda_model,corpus,id2word)
        #lda_model.print_topics()

## POS 

In [ ]:
class POS:

    def __init__(self,nlp):
        self.nlp = nlp

    def subject_predicate_object(self,doc):
        SPO = []
        for token in doc:
            #if the token is a verb
            if(token.pos_ == 'VERB'):
                subject = ""
                noun_chunks = [ n.lemma_ for n  in  doc.noun_chunks ]
                #only extract noun or pronoun subjects
                for sub_tok in token.lefts:
                    if(sub_tok.dep_ in ['nsubj','snubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN']): #,'PRON'
                        subject = sub_tok.lemma_
                        for sub_tok in token.rights:
                            #save the object in the phrase
                            if(sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                                resp = [subject,token.lemma_,sub_tok.lemma_]
                                steps = [0,2]
                                id_taken = []
                                for step in steps:
                                    for id,noun in enumerate(noun_chunks):
                                        if noun.find(resp[step])!=-1 and id not in id_taken:
                                            resp[step] = " ".join([tok.text for tok in nlp(noun) if tok.pos_ not in ["DET"]])
                                            id_taken.append(id)
                                            break

                                SPO.append(resp)
                                
                    
        return SPO


    def _noun_auxi_noun(self,doc,have=False):
        N_N = []
        aux = "be"
        if have:
            aux = "have"
        for token in doc:
            noun_chunks = [ n.lemma_ for n  in  doc.noun_chunks ]
            #if the token is a verb
            #if(token.pos_ == 'VERB'):
            if token.lemma_ == aux:
                subject = ""
                #only extract noun or pronoun subjects
                for sub_tok in token.lefts:
                    if sub_tok.pos_ in ['NOUN','PROPN']: #
                        subject = sub_tok.lemma_
                        for sub_tok in token.rights:
                            #save the object in the phrase
                            if sub_tok.pos_ in ['NOUN','PROPN']:
                                resp = [subject,token.lemma_,sub_tok.lemma_]
                                steps = [0,2]
                                id_taken = []
                                for step in steps:
                                    for id,noun in enumerate(noun_chunks):
                                        if noun.find(resp[step])!=-1 and id not in id_taken:
                                            resp[step] = " ".join([tok.text for tok in nlp(noun) if tok.pos_ not in ["DET"]])
                                            id_taken.append(id)
                                            break
 
                                N_N.append(resp)
        
        return N_N

    def noun_is_have_noun(self,doc):
        N_N = self._noun_auxi_noun(doc)
        N_N.extend(self._noun_auxi_noun(doc,True))
        return N_N

    #create data frame of predicate sentences :
    def dataframe_predicate(self,sentences):
        objects,subjects,predicates,sentences_ = [],[],[],[]
        noun1,verbs,noun2, sentences_h_i = [],[],[],[]
        for sent in sentences:
            doc = self.nlp(sent)
            SPO = self.subject_predicate_object(doc)
            N_N = self.noun_is_have_noun(doc)

            for nn in N_N:
                noun1.append(nn[0])
                verbs.append(nn[1])
                noun2.append(nn[2])
                sentences_h_i.append(sent)

            for spo in SPO:
                subjects.append(spo[0])
                predicates.append(spo[1])
                objects.append(spo[2])
                sentences_.append(sent)


        is_have = pd.DataFrame({'sentences': sentences_h_i, 'Noun1': noun1,'Is/Have':verbs, 'Noun2': noun2})
        
        is_ = is_have[is_have['Is/Have']=="be"]
        have = is_have[is_have['Is/Have']=="have"]

        is_["Is/Have"] = "is"
        have["Is/Have"] = "has"

        predicate = pd.DataFrame({'sentences': sentences_, 'subject': subjects,'predicate':predicates, 'object': objects})

        return {"predicate":predicate,"is":is_,"have":have}
                


## Matcher

In [ ]:
from spacy.matcher import Matcher

class Match:
    patterns = [

        [   
            {'DEP':'amod', 'OP':"?"}, # adjectival modifier
            {'POS': {"IN":['NOUN',"PROPN"]}},
            {'LOWER': 'such'},
            {'LOWER': 'as'},
            {'POS': {"IN":['NOUN',"PROPN"]}}
        ],
        [
            {'DEP':'amod', 'OP':"?"}, 
            {'POS':{"IN":['NOUN',"PROPN"]}}, 
            {'LOWER': 'and', 'OP':"?"}, 
            {'LOWER': 'or', 'OP':"?"}, 
            {'LOWER': 'other'}, 
            {'POS':{"IN":['NOUN',"PROPN"]}}
        ],
        [
            {'DEP':'nummod','OP':"?"}, # numeric modifier 
            {'DEP':'amod','OP':"?"}, # adjectival modifier 
            {'POS':{"IN":['NOUN',"PROPN"]}}, 
            {'IS_PUNCT': True}, 
            {'LEMMA': {"in":['include',"especially"]}}, 
            {'DEP':'nummod','OP':"?"}, 
            {'DEP':'amod','OP':"?"}, 
            {'POS':{"IN":['NOUN',"PROPN"]}}
        ],
    ]
        
    def __init__(self,nlp):
        self.nlp = nlp
        self.matcher = Matcher(self.nlp.vocab)
        self.matcher.add("m1",self.patterns)

    def df_matcher(self,sentences):
        x = []
        y = []
        relation = []
        rels = ["such as","and other","or other", "especially","include"]
        for sent in sentences:
            doc = self.nlp(sent)
            matches = self.matcher(doc)
            if matches:
                for match_id, start, end in matches:
                    for r in rels:
                        text = doc[start:end].lemma_
                        if text.find(r) != -1 :
                            split = text.split(r)
                            x.append(split[0])
                            relation.append(r)
                            y.append(split[1])

        return pd.DataFrame({"X":x,"Relation":relation,"Y":y}).drop_duplicates()
                    
    


## OWL

In [ ]:
from rdflib import Graph,URIRef,BNode,Literal,Namespace,Graph
from rdflib.namespace import FOAF,DCTERMS,DC,DOAP,OWL,RDFS,SKOS,VOID,XMLNS,XSD,RDF,SDO
from rdflib.extras import describer

class OWLearning:
    
    def __init__(self):

        self.g=Graph()
        self.g.bind('owl',OWL)
        self.g.bind('pr',"http://example.org/projet#")
        self.ns_url="http://example.org/projet#"
        self.g.add((URIRef('http://example.org/project'),RDF.type ,OWL.Ontology))
        self.process_URI = ""
        self.process_name = ""

        self.main_class_uri = URIRef(self.ns_url+"risk_management")
        self.g.add((self.main_class_uri,RDF.type,OWL.Class))

    def init(self,process_name):
        self.process_name = process_name.replace(" ","_")
        self.process_URI=URIRef(self.ns_url+self.process_name)
        self.g.add((self.process_URI,RDFS.subClassOf,self.main_class_uri))

        name="inputs_"+self.process_name
        uri=URIRef(self.ns_url+name)
        self.g.add((uri,RDFS.subClassOf,self.process_URI))

        name="outputs_"+self.process_name
        uri=URIRef(self.ns_url+name)
        self.g.add((uri,RDFS.subClassOf,self.process_URI))

        name="tools_and_techniques_"+self.process_name
        uri=URIRef(self.ns_url+name)
        self.g.add((uri,RDFS.subClassOf,self.process_URI))


    def add_concepts(self,df_concept):
        for i,concept in enumerate(df_concept["parent concept"]):
            uri = URIRef(self.ns_url+concept.replace(" ","_"))
            self.g.add((uri,RDFS.subClassOf,self.process_URI))
            for s_concept in df_concept["concept"][i]:
                uri_val = URIRef(self.ns_url+s_concept.replace(" ","_"))
                self.g.add((uri_val,RDF.type,OWL.NamedIndividual))
                self.g.add((uri_val,RDF.type,uri))

    def add_annotations(self,df_definition):
        for i,row in df_definition.iterrows():
            def_uri = URIRef(self.ns_url+row["definition"].replace("\n","_").replace(" ","_"))
            concept_uri = URIRef(self.ns_url+row["concept"].replace(" ","_"))
            #self.g.add((def_uri,RDF.type,OWL.Annotation))
            if bool(re.search(".*Des.*in|.*Def.*in",row["definition"])):
                 self.g.add((concept_uri,RDFS.seeAlso,def_uri))
            else:
                self.g.add((concept_uri,RDFS.isDefinedBy,def_uri))

            #self.g.add((def_uri,RDFS.isDefinedBy,concept_uri))

    def add_object_property(self,df_predicate):
        for i,row in df_predicate["predicate"].iterrows():
            pred_uri = URIRef(self.ns_url+row["predicate"].replace(" ","_"))
            self.g.add((pred_uri,RDF.type,OWL.ObjectProperty))
            sub_uri = URIRef(self.ns_url+row["subject"].replace(" ","_"))
            obj_uri = URIRef(self.ns_url+row["object"].replace(" ","_"))
            self.g.add((sub_uri,pred_uri,obj_uri))

    def add_data_property(self,df_predicate):
        for i,row in df_predicate["have"].iterrows():
            noun1=URIRef(self.ns_url+row["Noun1"].replace(" ","_"))
            ind1=URIRef(self.ns_url+row["Noun2"].replace(" ","_"))
            
            self.g.add((ind1,RDF.type,OWL.DatatypeProperty))
            self.g.add((noun1,RDFS.subClassOf,self.process_URI))###
            self.g.add((ind1,RDF.type,noun1))

    def add_individuals(self,input_tools_output,df_predicate):
        
        for name,values in input_tools_output.items():
            uri = URIRef(self.ns_url+(name+"_"+self.process_name).replace(" ","_"))
            for val in values:
                uri_val = URIRef(self.ns_url+val.replace(" ","_"))
                self.g.add((uri_val,RDF.type,OWL.NamedIndividual))
                self.g.add((uri_val,RDF.type,uri))

        for i,row in df_predicate["is"].iterrows():
            noun1=URIRef(self.ns_url+row["Noun1"].replace(" ","_"))
            ind1=URIRef(self.ns_url+row["Noun2"].replace(" ","_"))
            self.g.add((ind1,RDF.type,OWL.NamedIndividual))
            self.g.add((noun1,RDFS.subClassOf,self.process_URI))###
            self.g.add((ind1,RDF.type,noun1))


    def generate_owl(self,cognition,url='/content/drive/MyDrive/dataset/ai_cognition/onto1.owl'):
        for i,_ in enumerate(cognition.process[1:],1):
            cognition.set_process(i)
            self.init(cognition.get_process)
            self.add_concepts(cognition.df_concept)
            self.add_object_property(cognition.pos_predicate())
            self.add_data_property(cognition.pos_predicate())
            self.add_individuals(cognition.inputs_tools_outputs(i),cognition.pos_predicate())
            self.add_annotations(cognition.df_definition)
        with open(url, 'w', encoding='utf-8') as output:
            output.write(self.g.serialize(format="pretty-xml"))


## Cognition Base

In [ ]:
import textacy
from functools import partial

class CognitionBase:

    def __init__(self,nlp):
        self.nlp = nlp
        self.eval = Evaluator()
        self.pos = POS(nlp)

    def _segment_sentences(self,text):
        #text = ProcessText.remove_double_space(text.replace("\t"," ")) #replace("\n"," ").
        return [ sentence.text for sentence in self.nlp(text).sents if len(sentence)> 1]

    def _clean_sentences(self,sentences):
        for i,sent in enumerate(sentences):
            sentence = ProcessText.clean_text(sent)
            sentence = ProcessText.remove_double_space(sentence.replace("\n"," "))
            if len(sentence) > 1:
                sentences[i] = sentence

        return sentences

    def _tokens(self,sentences="",text="",all_pos=False,nlp_tokens=False,lemma=True):
        tokens = self.__tokenization(sentences,text,all_pos)
        if lemma :
            return [tok.lemma_.lower() for tok in tokens]
        if nlp_tokens:
            return tokens

        return self.__tokens(tokens)

    def __tokenization(self,sentences="",text="",all_pos=False):
        """Segment a sentence in POS_TAG for each document

        Args:
            all_pos (bool, optional): if you want to extract a specifiques pos_tags. Defaults to False.

        Returns:
            Tokens: dic of tokens per document
        """ 
        if sentences!="":
            text = " ".join(sentences)

        if not all_pos:
            pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB', "ADV"]  
            return [token for token in self.nlp(text) if not token.is_stop and len(token)>1 and token.pos_ in pos_tag]

        return[token for token in self.nlp(text) if not token.is_stop and len(token)>1 ]

    def __tokens(self,nlp_tokens):
        return [tok.text.lower() for tok in nlp_tokens]

    def _topic_process(self,tokens_lemma):
        topics,_ = self.eval.LDA(tokens_lemma)
        return topics

    def _all_lemma(self,lemma=True,nlp_tokens=False):
        lemmas = []
        for process in self.process:
            sentences = self._segment_sentences(process)
            sentences = self._clean_sentences(sentences)
            lemmas.append(self._tokens(sentences=sentences,nlp_tokens=nlp_tokens,lemma=lemma))

        return lemmas

    def _synonym(self,token,list_tokens,sy_):
        """Parse synonym in the document for a given token

        Args:
            token (string): token to find synonym
            list_tokens (list): list of tokens

        Returns:
            list: synonyms
        """        
        s = []
        syn = []
        for vayn in wordnet.synsets(token):
            for tok in vayn.lemmas():
                 s.append(tok.name())
        for i in list_tokens:
            if i in s:
                syn.append(i)
        sy_.extend(syn)
        return list(set(syn)),list(set(sy_))

    def _n_grams(self,doc,n=1):
        return list(textacy.extract.terms(doc,ngs=partial(textacy.extract.ngrams, n=n, include_pos={"NOUN"}), dedupe=False))

    def _vna(self):
        adj,verbs,nouns = [],[],[]
        segments = []
        for tokens in self._all_lemma(lemma=False,nlp_tokens=True):
            segments.extend(tokens)
        for token in segments:
            if token.pos_ == "ADJ":
                adj.append(token.lemma_.lower())
            elif token.pos_ == "VERB":
                verbs.append(token.lemma_.lower())
            elif token.pos_ in ["NOUN","PROPN"]:
                nouns.append(token.lemma_.lower())
        return {"verbs":list(set(verbs)),"nouns":list(set(nouns)),"adjs":list(set(adj))}

## SparQL

In [ ]:
class OwlQuery:

    def __init__(self, owl_path) -> None:
        self.g = Graph()
        self.g.parse(owl_path)

    def _parse(self, data: str) -> str:
        return data.split(":")[1].replace("_", " ")

    def get_process(self) -> list:

        res = self.g.query(
            'SELECT DISTINCT ?process ?clas WHERE {  ?clas rdfs:subClassOf    ?process .FILTER( STR(?clas) = "http://example.org/projet#process")} ORDER BY ?process')
        return [self._parse(i.process.n3(self.g.namespace_manager)) for i in res]

    def get_inp_tools_oup(self, process_name, data_type) -> list:
        data = data_type.replace(" ", "_")+"_"+process_name.replace(" ", "_")
        res = self.g.query(
            'SELECT DISTINCT ?input ?clas WHERE {  ?clas  rdf:type  ?input .FILTER( STR(?input) = "http://example.org/projet#'+data+'") }')
        return [self._parse(i.clas.n3(self.g.namespace_manager)).replace(data_type, "") for i in res]

    def _init_collection(self, id, collection_, is_dict=False):
        collection_[id] = collection_[
            id] if id in collection_.keys() else ({} if is_dict else [])
        return collection_

    def get_process_by_type(self, data_type):
        res = self.g.query(" SELECT DISTINCT ?type ?clas ?process   " +
                           " WHERE { ?clas  rdf:type   ?type." +
                           "FILTER contains( STR(?type) ,( \"http://example.org/projet#"+data_type+"_\"))" +
                           "}" +
                           "ORDER BY ?type")

        return [self._parse(i.clas.n3(self.g.namespace_manager)).replace(data_type, "") for i in res]

    def _get_other_info(self, data_type):
        data = []
        res = self.g.query("SELECT DISTINCT ?type ?clas ?process   " +
                           " WHERE { ?clas  rdfs:"+data_type+"   ?type." +
                           "}")
        for i in res:
            defi = self._parse(i.type.n3(self.g.namespace_manager)).split("#")
            data.append(
                (self._parse(i.clas.n3(self.g.namespace_manager)), defi[1] if len(defi) > 1 else defi[0]))
        return data

    def get_belong(self, instance):
        instance = instance.replace(" ", "_")

        res = self.g.query("SELECT DISTINCT ?p ?concept ?instance  " +
                           " WHERE { ?concept rdf:type  ?p." +
                           "?p rdfs:subClassOf  ?instance." +
                           "FILTER contains( STR(?concept) ,( \"http://example.org/projet#"+instance+"\"))" +
                           "}" +
                           "ORDER BY ?concept")
        return list(set([self._parse(i.instance.n3(self.g.namespace_manager)) for i in res]))

    def get_other_info(self):
        data = {}
        infos = self._get_other_info("isDefinedBy")
        infos.extend(self._get_other_info("seeAlso"))

        for word, defi in infos:
            data = self._init_collection(word, data)
            data[word].append(defi)

        return data

    def data(self, process_name):
        annotations = self.get_other_info()

        data = {pro: {} for pro in self.get_process()}
        for process, _ in data.items():
            for data_type in ["inputs", "tools_and_techniques", "outputs"]:
                data[process][data_type] = {
                    i: {} for i in self.get_inp_tools_oup(process, data_type)
                }

        for process, d in data.items():  # 1
            search = False
            if process_name == process:
                search = True
            if process in annotations.keys():
                data[process]["definition"] = " ".join(annotations[process])
            for data_type, values in d.items():  # 2
                if data_type == "definition":
                    continue
                for value, _ in values.items():  # 3
                    key = "Other informations"
                    data[process][data_type][value] = self._init_collection(
                        key, data[process][data_type][value])
                    if value in annotations.keys():
                        data[process][data_type][value][key].append(
                            " ".join(annotations[value]))
                    if search:
                        key = f"{data_type} belong to"
                        data[process][data_type][value] = self._init_collection(
                            key, data[process][data_type][value])
                        data[process][data_type][value][key] = self.get_belong(
                            value)
                    for other_process, other_d in data.items():  # 1
                        if other_process != process:
                            for other_data_type, other_values in other_d.items():  # 2
                                if other_data_type in ["inputs", "tools_and_techniques", "outputs"] and value in other_values.keys():
                                    key = "Is "+other_data_type+" of these process"
                                    data[process][data_type][value] = self._init_collection(
                                        key, data[process][data_type][value])
                                    data[process][data_type][value][key].append(
                                        other_process)

        return data


## Cognition 

In [ ]:
from collections import Counter
from spacy import displacy 
import numpy as np

class Cognition(CognitionBase):

    _sentences = None
    _seg_tokens = None
    _cleaned_sentences = None
    _all_cleaned_sentences = None
    _all_pos = False
    process_i = 0
    _main_tokens = []
    _process_list = {1:"plan risk management",2:"identify risks",3:"perform qualitative risk analysis",4:"perform quantitative risk analysis",5:"plan risk responses",6:"control risks"}

    copyrights = ["©2013 Project Management Institute. A Guide to the Project Management Body of Knowledge (PMBOK® Guide) – Fifth Edition","©2009 Project Management Institute. Practice Standard for Project Risk Management"]

    def __init__(self,filenames,nlp,range_process=[[(335,338),(339,344),(345,353),(354,359),(360,367),(368,375),(376,380)],[(25,29),(31,35),(25,41),(31,47),(37,53),(43,61),(51,67)]],ranges=[(309,354),(13,56)]):
        CognitionBase.__init__(self,nlp)

        self.n_process = len(range_process[0])
        self.pdfs = [ParsePDF(filename) for filename in filenames]

        self.process = ["" for _ in range(len(range_process[0]))]

        for doc_i,ranges_ in enumerate(range_process):
            for process_i,range_ in enumerate(ranges_):
                content = self.pdfs[doc_i].content(range_[0],range_[1])
                for copyright in self.copyrights:
                    content = content.replace(copyright,"").replace("fi ","fi")
                self.process[process_i] += "\n"+content

        self.matcher = Match(nlp)

    @property
    def get_process(self):
        return self._process_list[self.process_i] if self.process_i > 0 else "Introduction"

    def set_process(self,num):
        self.process_i = num
        self._sentences = None
        self._seg_tokens = None
        self._cleaned_sentences = None
        self._all_pos = False

    def topics(self):
        tokens = [self._tokens(text=process ,lemma=True) for process in self.process]    
        return self._topic_process(tokens)

    @property
    def df_definition(self):
        keys = []
        for val in self.inputs_tools_outputs(self.process_i).values():
            keys.extend(val)

        data = {"concept":[],"definition":[]}
        for i,sent in enumerate(self.sentences):
            if len(re.findall("\d{2,}\.\d+.\d+.\d+ ("+"|".join(keys)+")",sent.lower()))>0:  
                match = re.findall("\d{2,}\.\d+.\d+.\d+.*",sent)[0].lower()
                match = re.sub("\d{2,}\.\d+.\d+.\d+ ","",match)
                definition = re.sub("\d{2,}\..*\n|\d{2,}.*\n","",sent)
                data["concept"].append(match)
                data["definition"].append(definition)

            if len(re.findall("\d{2,}\.\d+ "+self.get_process,sent.lower()))>0:
                definition = re.sub("\d{2,}\..*\n|\d{2,}.*\n","",sent)
                definition = "".join( definition.split("\n"))
                data["concept"].append(self.get_process)
                data["definition"].append(definition)
            
        return pd.DataFrame(data)

    @property
    def df_concept(self):
        doc = self.nlp(". ".join(self.cleaned_sentences))
        nouns_to_include = self.verbs_nouns_adj["nouns"]

        terms = self._n_grams(doc)
        p_concepts = list(set([ tok.lemma_ for tok in terms if len(tok.lemma_)>2 and tok.lemma_ in nouns_to_include]))

        data = pd.DataFrame({"parent concept":p_concepts})

        terms = self._n_grams(doc,n=2)
        terms.extend(self._n_grams(doc,n=3))

        terms = list(set([ ProcessText.clean_text(tok.lemma_) for tok in terms if (len(ProcessText.clean_text(tok.lemma_))>2 )])) 
        #and len(re.findall(tok.lemma_,doc.text))>0
        concepts = []
        new = []
        for concept in p_concepts:
            for conc in terms:
                if conc.find(concept) != -1:
                    if conc != concept:
                        new.append(conc)
            concepts.append(list(set(new)))
            new = []

        data["concept"] = concepts

        return data[data['concept'].map(len) > 0].reset_index(drop=True)

    @property
    def all_cleaned_sentences(self):
        if self._all_cleaned_sentences != None:
            return self._all_cleaned_sentences
        sentences = []
        for process in self.process:
             sentences.extend(self._clean_sentences(self._segment_sentences(process)))

        self._all_cleaned_sentences = sentences 
        return self._all_cleaned_sentences

    def maching_rules(self):
        return self.matcher.df_matcher(self.cleaned_sentences)

    @property
    def cleaned_sentences(self):
        """ remove numbers and punctuations

        Returns:
            [dic]: [text cleans]
        """   
        if self._cleaned_sentences != None :
            return self._cleaned_sentences

        cleaned_texts = self._clean_sentences(self.sentences)    

        self._cleaned_sentences = cleaned_texts
        return cleaned_texts

    @property
    def sentences(self):
        if self._sentences != None :
            return self._sentences

        data = self._segment_sentences(self.process[self.process_i])

        self._sentences = data
        return data
        
    @property
    def synonyms(self):
        """extract synonyms form given documents

        Returns:
            list: synonyms
        """        
        tokens = self._vna()

        all_synonyms = {}
        for pos,tokens_pos in tokens.items():
            sy_ = []
            synonyms = {}
            for token in tokens_pos:
                if token in sy_:
                    continue
                syns, sy_ = self._synonym(token,tokens_pos,sy_)
                if len(syns) > 1:
                    synonyms[token] = syns
            if len(synonyms)!=0:
                all_synonyms[pos] = list(synonyms.values())

        return all_synonyms

    def inputs_tools_outputs(self,process_i=None):
        if process_i != None:
            self.set_process(process_i)
        data = {"inputs":[],"tools and techniques":[],"outputs":[]}
        to_extracts = ["inputs","tools and techniques","outputs"]
        text = " ".join(self.sentences).lower()
        if self.process_i == 0:
            return data
        process_name = self._process_list[self.process_i]

        try:
            for to_extract in to_extracts:
                process = re.findall(f"\d+\.\d+\.\d+ {process_name}: {to_extract}",text)[0]
                process_number = re.findall(f"\d+\.\d+\.\d+",process)[0]
                elements = re.findall(f"{process_number}.\d+.*",text)
                elements = [re.sub("\d+\.\d+\.\d+\.\d+\s?","",el) for el in elements]
                data[to_extract].extend(elements)
        except:
            print("pattern not found")

        return data

    @property
    def main_tokens(self):
        if self._main_tokens != []:
            return self._main_tokens
        lemmas = self._all_lemma()
        documents = [ " ".join(tokens_) for tokens_ in lemmas ]

        df = self.eval.computeTFIDF(documents)
        self._main_tokens = list(df[(df > 0).all(axis=1)].index)
        return self._main_tokens

    @property
    def tokens(self):
        """List of token for each document

        Returns:
            dict: dictionary of list of token
        """        

        return [token.text.lower() for token in  self.segmentation() ]

    def show_dependancies(self): 
        sentences = self.sentences
        i_doc = np.random.randint(0,len(sentences)+1)
        i_sentence = np.random.randint(0,len(sentences[i_doc])+1)
        displacy.render(self.nlp(sentences[i_doc][i_sentence]), style='dep',jupyter=True)

    def segmentation(self,all_pos=False):
        """Segment a sentence in POS_TAG for each document

        Args:
            all_pos (bool, optional): if you want to extract a specifiques pos_tags. Defaults to False.

        Returns:
            Tokens: dic of tokens per document
        """ 
        if self._seg_tokens != None and self._all_pos == all_pos:
            return self._seg_tokens

        text = " ".join(self.cleaned_sentences)
        tokens = self._tokens(text=text,all_pos=all_pos,nlp_tokens=True,lemma=False) 
        
        self._all_pos = all_pos
        self._seg_tokens = tokens
        return tokens

    @property
    def lemmatization(self):
        return [token.lemma_ for token in self.segmentation()]

    @property
    def verbs_nouns_adj(self):
        main_tokens = self.main_tokens
        adj,verbs,nouns = [],[],[]
        for token in self.segmentation():
            if token.pos_ == "ADJ":
                if token.lemma_.lower() in main_tokens:
                    adj.append(token.lemma_.lower())
            elif token.pos_ == "VERB":
                if token.lemma_.lower() in main_tokens:
                    verbs.append(token.lemma_.lower())
            elif token.pos_ in ["NOUN","PROPN"]:
                if token.lemma_.lower() in main_tokens:
                    nouns.append(token.lemma_.lower())
        data = {"verbs":list(set(verbs)),"nouns":list(set(nouns)),"adjs":list(set(adj))}
        print("Process " +str(self.get_process) + " has  " + str(len(data["verbs"])) +" unique verbs and " + str(len(data["nouns"])) +" unique nouns and "+ str(len(data["adjs"])) +" unique adjectives." )
        return data

    @property
    def entities(self):
        entities = {}
        text = " ".join(self.cleaned_sentences)
        for ent in self.nlp(text).ents :
            if ent.label_ in ["ORG"]:
                entities[ent.text] = ent.label_
        
        return entities

    def word_freq(self): 
        """Count the occurrences of words for each document

        Returns:
            list: [description]
        """        
        word_freq = Counter(self.tokens)
        return word_freq.most_common()

    def pos_predicate(self):
        return self.pos.dataframe_predicate(self.cleaned_sentences)

    def evaluate(self):
        """Evaluate the documents according metrics
        """    
        data = {}  
        lemmas =  self._all_lemma()
        documents = [ " ".join(tokens_) for tokens_ in self._all_lemma()]

        print( "Evaluation TFIDF")
        df = self.eval.computeTFIDF(documents)
        data["tfidf"] = df
        print(df.head(30))

        print("\n")
        print( "Evaluation Cosine")
        df = self.eval.similarity(documents)
        data["cosine"] = df
        print(df)
        '''
        model_lda,vis = self.eval.LDA(lemmas)
        print("\n")
        print( "Evaluation LDA")
        data["lda"] = [model_lda,vis]
        print(model_lda.print_topics())
        '''
        return data


#Test

In [ ]:
 cog = Cognition(["PMBOK 5th.pdf","PMI.pdf"],nlp,)

In [ ]:
cog.set_process(2)

## Definitions

In [ ]:
cog.df_definition

,concept,definition
0,perform qualitative risk analysis,Perform Qualitative Risk Analysis is the proc...
1,risk management plan,\nDescribed in Section 11.1.3.1.
2,scope baseline,\nDescribed in Section 5.4.3.1.
3,risk register,\n \nDescribed in Section 11.2.3.1.
4,enterprise environmental factors,\nDescribed in Section 2.1.5.
5,organizational process assets,\nDescribed in Section 2.1.4.
6,risk probability and impact assessment,\nRisk probability assessment investigates the...
7,probability and impact matrix,\n \nRisks can be prioritized for further quan...
8,risk data quality assessment,\nRisk data quality assessment is a technique ...
9,risk categorization,\nRisks to the project can be categorized by s...


## Extraction

In [ ]:
".".join(cog.sentences)

"\n328\n \n11 - PROJECT RISK MANAGEMENT\n11.3 Perform Qualitative risk Analysis\nPerform Qualitative Risk Analysis is the process of prioritizing risks for further analysis or action by assessing \nand combining their probability of occurrence and impact..The key beneft of this process is that it enables project \nmanagers to reduce the level of uncertainty and to focus on high-priority risks..The inputs, tools and techniques, \nand outputs of this process are depicted in Figure 11-8..Figure 11-9 depicts the data fow diagram of the process..\nInputs\nTools & Techniques\nOutputs\n.1 Risk management plan\n.2 Scope baseline\n.3 Risk register\n.4 Enterprise environmental \n \nfactors\n.5 Organizational process \n \nassets\n.1 Risk probability and \n \nimpact assessment\n.2.Probability and impact \n \nmatrix\n.3 Risk data quality \n \nassessment\n.4 Risk categorization\n.5 Risk urgency assessment\n.6.Expert judgment\n.1 Project documents \n \nupdates\nFigure 11-8..Perform Qualitative risk A

## Data cleaning

In [ ]:
"".join(cog.cleaned_sentences)

" Perform Qualitative Risk Analysis is the process of prioritizing risks for further analysis or action by assessing and combining their probability of occurrence and impact.The key beneft of this process is that it enables project managers to reduce the level of uncertainty and to focus on high-priority risks.The inputs, tools and techniques, and outputs of this process are depicted in Figure -Figure - depicts the data fow diagram of the process. Inputs Tools & Techniques Outputs . Risk management plan . Scope baseline . Risk register . Enterprise environmental factors . Organizational process assets . Risk probability and impact assessment .Probability and impact matrix . Risk data quality assessment . Risk categorization . Risk urgency assessment .Expert judgment . Project documents updates Figure -Perform Qualitative risk Analysis: Inputs, tools & techniques, and outputs Project Risk Management Perform Qualitative Risk Analysis Plan Risk Management Identify Risks • Project document

## Evaluation

In [ ]:
data = cog.evaluate()

Evaluation TFIDF
                 TFIDF_1   TFIDF_2   TFIDF_3  ...   TFIDF_5   TFIDF_6   TFIDF_7
11              0.000000  0.000000  0.000000  ...  0.000000  0.003191  0.004461
ability         0.000000  0.003504  0.007937  ...  0.006972  0.003653  0.005106
able            0.000000  0.000000  0.000000  ...  0.006943  0.010912  0.011440
academic        0.000000  0.000000  0.004177  ...  0.000000  0.000000  0.000000
accept          0.010516  0.009275  0.005251  ...  0.002050  0.016112  0.011261
acceptable      0.003505  0.006183  0.005251  ...  0.004101  0.004834  0.002252
acceptance      0.000000  0.003981  0.002254  ...  0.000000  0.012448  0.005800
accepted        0.000000  0.000000  0.000000  ...  0.000000  0.003845  0.000000
access          0.000000  0.003981  0.002254  ...  0.000000  0.002075  0.002900
accommodate     0.000000  0.000000  0.000000  ...  0.000000  0.006383  0.000000
accomplish      0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
accomplishment  0.00000

In [ ]:
data["lda"][1]

KeyError: ignored

## LDA

In [ ]:
cog.topics().print_topics()

[(0,
  '0.063*"risk" + 0.033*"project" + 0.013*"management" + 0.012*"plan" + 0.010*"process" + 0.009*"analysis" + 0.007*"response" + 0.006*"quantitative" + 0.006*"objective" + 0.006*"identify"'),
 (1,
  '0.030*"risk" + 0.021*"project" + 0.018*"management" + 0.009*"process" + 0.008*"response" + 0.008*"plan" + 0.007*"identify" + 0.006*"analysis" + 0.004*"level" + 0.003*"perform"'),
 (2,
  '0.099*"risk" + 0.045*"project" + 0.024*"response" + 0.022*"analysis" + 0.020*"plan" + 0.016*"process" + 0.015*"management" + 0.012*"quantitative" + 0.010*"cost" + 0.009*"objective"'),
 (3,
  '0.108*"risk" + 0.063*"project" + 0.051*"management" + 0.027*"process" + 0.023*"plan" + 0.014*"identify" + 0.011*"stakeholder" + 0.009*"analysis" + 0.008*"technique" + 0.007*"objective"'),
 (4,
  '0.076*"risk" + 0.029*"project" + 0.022*"analysis" + 0.016*"plan" + 0.016*"process" + 0.011*"response" + 0.011*"management" + 0.010*"cost" + 0.009*"quantitative" + 0.008*"identify"'),
 (5,
  '0.035*"risk" + 0.019*"project"

## Tokenization

In [ ]:
cog.main_tokens

['accept',
 'acceptable',
 'account',
 'action',
 'activity',
 'address',
 'agree',
 'analysis',
 'analyze',
 'apply',
 'approach',
 'appropriate',
 'arise',
 'assess',
 'assessment',
 'asset',
 'assign',
 'assumption',
 'available',
 'base',
 'baseline',
 'basis',
 'benefit',
 'beneft',
 'breakdown',
 'case',
 'cause',
 'change',
 'chapter',
 'characteristic',
 'clearly',
 'combination',
 'common',
 'communicate',
 'communication',
 'condition',
 'consider',
 'contingency',
 'control',
 'copy',
 'corresponding',
 'cost',
 'critical',
 'cycle',
 'datum',
 'define',
 'definition',
 'defne',
 'degree',
 'depend',
 'describe',
 'detail',
 'determine',
 'develop',
 'diagram',
 'diego',
 'distribution',
 'document',
 'early',
 'effect',
 'effective',
 'element',
 'ensure',
 'evaluate',
 'evaluation',
 'event',
 'example',
 'exist',
 'expert',
 'exposure',
 'factor',
 'figure',
 'focus',
 'follow',
 'frequency',
 'fuentes',
 'gathering',
 'give',
 'group',
 'identify',
 'impact',
 'importanc

## Segmentation & synonyms

In [ ]:
for i,token in enumerate(list(cog.segmentation(True))):
    print (f'{token.text:{15}} {token.lemma_:{15}} {token.pos_:{10}} {token.tag_}')
    if i > 20:
        break

Perform         perform         VERB       VBP
Qualitative     qualitative     ADJ        JJ
Risk            risk            NOUN       NN
Analysis        Analysis        PROPN      NNP
process         process         NOUN       NN
prioritizing    prioritize      VERB       VBG
risks           risk            NOUN       NNS
analysis        analysis        NOUN       NN
action          action          NOUN       NN
assessing       assess          VERB       VBG
combining       combine         VERB       VBG
probability     probability     NOUN       NN
occurrence      occurrence      NOUN       NN
impact          impact          NOUN       NN
key             key             ADJ        JJ
beneft          beneft          NOUN       NN
process         process         NOUN       NN
enables         enable          VERB       VBZ
project         project         NOUN       NN
managers        manager         NOUN       NNS
reduce          reduce          VERB       VB
level           level     

In [ ]:
cog.show_dependancies()

In [ ]:
cog.synonyms

{'adjs': [['apparent', 'evident'],
  ['minor', 'low', 'small', 'little'],
  ['recent', 'later', 'previous'],
  ['good', 'near', 'close'],
  ['suitable', 'desirable'],
  ['special', 'specific', 'exceptional', 'particular'],
  ['undifferentiated', 'uniform', 'consistent'],
  ['external', 'outside'],
  ['achievable', 'manageable'],
  ['historic', 'historical'],
  ['primary', 'main', 'principal', 'independent'],
  ['single', 'exclusive'],
  ['unanticipated', 'unforeseen'],
  ['quick', 'immediate', 'prompt'],
  ['likely', 'possible', 'potential'],
  ['underlying', 'fundamental', 'inherent'],
  ['light', 'weak'],
  ['medium', 'sensitive'],
  ['clear', 'subject', 'open'],
  ['critical', 'vital'],
  ['medium', 'intermediate', 'average'],
  ['genuine', 'actual'],
  ['unmanageable', 'difficult'],
  ['honest', 'good'],
  ['unacceptable', 'impossible'],
  ['lead', 'left'],
  ['crucial', 'essential', 'necessary'],
  ['wide', 'broad'],
  ['lead', 'direct'],
  ['coherent', 'consistent'],
  ['sure', '

## NER

In [ ]:
(cog.entities)

{'Inputs Tools & Techniques Outputs': 'ORG',
 'PMBOK': 'ORG',
 'PMI': 'ORG',
 'Plan Schedule Management Enterprise/ Organization Project Documents Figure - Perform Quantitative risk Analysis': 'ORG',
 'Project Risk Management': 'ORG',
 'Project Risk Management Perform Qualitative Risk Analysis Plan Risk Management Identify Risks • Project': 'ORG',
 'Project Risk Management Perform Quantitative Risk Analysis Plan Risk Management Identify Risks • Project': 'ORG',
 'RBS': 'ORG',
 'Schedule': 'ORG',
 'Section': 'ORG',
 'The Perform Qualitative Risk Analysis': 'ORG',
 'The Perform Quantitative Risk Analysis': 'ORG',
 'WBS': 'ORG',
 'the Identify Risks and Perform Qualitative Analysis': 'ORG',
 'the Organization’s Objectives': 'ORG',
 'the Perform Qualitative Risk Analysis': 'ORG',
 'the Plan Risk Management': 'ORG',
 'the Project Risk Management': 'ORG'}

## Occurrence

In [ ]:
cog.word_freq()

[('risk', 331),
 ('project', 183),
 ('management', 133),
 ('risks', 115),
 ('process', 77),
 ('plan', 64),
 ('analysis', 59),
 ('probability', 34),
 ('qualitative', 33),
 ('impact', 32),
 ('stakeholders', 32),
 ('perform', 28),
 ('information', 28),
 ('objectives', 26),
 ('identify', 24),
 ('high', 23),
 ('figure', 22),
 ('organization', 22),
 ('tools', 19),
 ('techniques', 19),
 ('assessment', 19),
 ('priority', 18),
 ('time', 18),
 ('responses', 18),
 ('identified', 18),
 ('data', 16),
 ('objective', 16),
 ('low', 16),
 ('identification', 16),
 ('level', 15),
 ('register', 15),
 ('quantitative', 15),
 ('processes', 15),
 ('pmi', 14),
 ('cost', 14),
 ('required', 14),
 ('activities', 14),
 ('planning', 14),
 ('use', 14),
 ('scope', 13),
 ('factors', 13),
 ('considered', 13),
 ('success', 13),
 ('matrix', 12),
 ('quality', 12),
 ('corresponding', 12),
 ('occur', 12),
 ('team', 12),
 ('uncertainty', 11),
 ('common', 11),
 ('effect', 11),
 ('order', 11),
 ('specific', 11),
 ('ensure', 11

## Nouns/Verbs

In [ ]:
data = cog.verbs_nouns_adj
df = [pd.DataFrame({"Verbs":data["verbs"]}),pd.DataFrame({"Nouns":data["nouns"]})]
pd.concat(df, axis=1).fillna("")

Process perform qualitative risk analysis has  67 unique verbs and 122 unique nouns and 29 unique adjectives.


,Verbs,Nouns
0,approach,quality
1,agree,resource
2,require,scope
3,ensure,action
4,plan,management
...,...,...
117,,source
118,,reproduction
119,,frequency
120,,support


In [ ]:
data["verbs"]

['approach',
 'agree',
 'require',
 'ensure',
 'plan',
 'indicate',
 'arise',
 'need',
 'determine',
 'manage',
 'apply',
 'depend',
 'scope',
 'select',
 'benefit',
 'risk',
 'evaluate',
 'address',
 'license',
 'limit',
 'change',
 'understand',
 'give',
 'accept',
 'present',
 'exist',
 'review',
 'identify',
 'register',
 'include',
 'develop',
 'detail',
 'repeat',
 'result',
 'take',
 'set',
 'use',
 'increase',
 'show',
 'defne',
 'consider',
 'occur',
 'communicate',
 'document',
 'output',
 'record',
 'follow',
 'represent',
 'incorporate',
 'work',
 'describe',
 'impact',
 'prioritize',
 'assess',
 'relate',
 'perform',
 'provide',
 'assign',
 'support',
 'memberid',
 'base',
 'report',
 'analyze',
 'focus',
 'lead',
 'recognize',
 'define']

## Spacy Rule Based Matching

In [ ]:
cog.maching_rules()

,X,Relation,Y
0,team,and other,stakeholder
1,objective,such as,schedule
2,"explanatory detail,",include,assumption
3,"detail,",include,assumption
4,management,and other,stakeholder
6,member,and other,stakeholder
7,parameter,and other,input
9,"tool,",include,interview


## POS (Part Of Speech) Tagging Rules Extraction

In [ ]:
data = cog.pos_predicate()

In [ ]:
data["is"]

,sentences,Noun1,Is/Have,Noun2
0,"This copy is a PMI Member benefit, not for dis...",copy,is,PMI Member benefit
1,"This copy is a PMI Member benefit, not for dis...",copy,is,PMI Member benefit
2,"This copy is a PMI Member benefit, not for dis...",copy,is,PMI Member benefit
3,"This copy is a PMI Member benefit, not for dis...",copy,is,PMI Member benefit
4,Risk data quality assessment is a technique t...,risk datum quality assessment,is,technique
5,The values used in the example in Figure - are...,figure,is,representative
6,"This copy is a PMI Member benefit, not for dis...",copy,is,PMI Member benefit
7,"This copy is a PMI Member benefit, not for dis...",copy,is,PMI Member benefit
8,"This copy is a PMI Member benefit, not for dis...",copy,is,PMI Member benefit
9,In the same way that project management is a ...,project management,is,process


In [ ]:
data["have"]

,sentences,Noun1,Is/Have,Noun2
11,Each category of risk identification techniqu...,category,has,strength
16,This is useful since it is common for risks to...,risk,has,uneven impact


In [ ]:
data["predicate"]

,sentences,subject,predicate,object
0,Figure - depicts the data fow diagram of the p...,figure,depict,datum fow diagram
1,Expert judgment . Project documents updates Fi...,project document,update,figure
2,"Perform Qualitative risk Analysis: Inputs, too...",Project Risk Management,identify,Risks
3,"Perform Qualitative risk Analysis: Inputs, too...",Project document,update,risk management plan
4,"Perform Qualitative risk Analysis: Inputs, too...",scope baseline,create,WBS enterprise/ Organization Project Documents...
...,...,...,...,...
74,"Therefore, data collection and evaluation tool...",datum collection and evaluation tool,require,management support
75,This capability provides a list of risks that ...,capability,provide,list
76,This is useful since it is common for risks to...,risk,have,uneven impact
77,A combination of the risk analysis information...,area,exhibit,risk analysis information


## Input-tools-Outputs

In [ ]:
cog.inputs_tools_outputs()

pattern not found


{'inputs': [], 'outputs': [], 'tools and techniques': []}

## Concept Extration


In [ ]:
cog.df_concept

Process perform qualitative risk analysis has  67 unique verbs and 122 unique nouns and 29 unique adjectives.


,parent concept,concept
0,quality,"[quality risk, quality information, quality as..."
1,resource,"[risk management resource, management resource]"
2,scope,"[project scope, project scope statement, scope..."
3,action,[management action]
4,management,"[cost management, risk management procedure, r..."
...,...,...
73,view,"[level view, review approach]"
74,representation,[representation technique]
75,source,"[risk management resource, management resource]"
76,support,[management support]


## OWL

In [ ]:
OWLearning().generate_owl(cog)

Process plan risk management has  65 unique verbs and 119 unique nouns and 26 unique adjectives.
Process identify risks has  72 unique verbs and 120 unique nouns and 28 unique adjectives.
Process perform qualitative risk analysis has  67 unique verbs and 122 unique nouns and 29 unique adjectives.
Process perform quantitative risk analysis has  66 unique verbs and 127 unique nouns and 30 unique adjectives.


http://example.org/projet#"_chance_node does not look like a valid URI, trying to serialize this will break.


Process plan risk responses has  71 unique verbs and 131 unique nouns and 32 unique adjectives.
Process control risks has  70 unique verbs and 127 unique nouns and 28 unique adjectives.


## SparkQL

In [ ]:
OwlQuery('/content/drive/MyDrive/dataset/ai_cognition/onto1.owl').data("control risks")

http://example.org/projet#"_chance_node does not look like a valid URI, trying to serialize this will break.


{'control risks': {'inputs': {'project management plan': {'Is inputs of these process': ['plan risk management'],
    'Other informations': [' In planning risk management, all approved subsidiary management plans and baselines should be taken into  consideration in order to make the risk management plan consistent with them.>  Described in Section 4.2.3.1.'],
    'inputs belong to': ['plan risk management',
     'control risks',
     'identify risks',
     'perform quantitative risk analysis',
     'perform qualitative risk analysis',
     'plan risk responses']},
   'risk register': {'Is inputs of these process': ['perform qualitative risk analysis',
     'perform quantitative risk analysis',
     'plan risk responses'],
    'Is outputs of these process': ['identify risks'],
    'Other informations': ['   The primary output from Identify Risks is the initial entry into the risk register.    Described in Section 11.2.3.1.  Described in Section 11.2.3.1.'],
    'inputs belong to': ['pla